In [81]:
import psycopg2

def create_db(conn, cursor):
    cursor.execute("""
            CREATE TABLE IF NOT EXISTS client
            (
                id SERIAL PRIMARY KEY,
                first_name VARCHAR(40) NOT NULL,
                last_name VARCHAR(40) NOT NULL,
                email VARCHAR(60) NOT NULL
            );
            """)
    
    cursor.execute("""
            CREATE TABLE IF NOT EXISTS client_phone
            (
                id SERIAL PRIMARY KEY,
                client_id INTEGER REFERENCES client(id),
                phone VARCHAR(25) NULL
            );
            """)
    conn.commit()
    return "Таблицы успешно создались"

def add_client(conn, cursor, first_name, last_name, email):
    cursor.execute("""
        INSERT INTO client(first_name, last_name, email) VALUES(%s, %s, %s) RETURNING id, first_name, last_name, email;
        """, (first_name, last_name, email))
    conn.commit()
    return cursor.fetchall()

def add_phone(conn, cursor, client_id, phone):
    cursor.execute("""
        INSERT INTO client_phone(client_id, phone) VALUES(%s, %s) RETURNING id, client_id, phone;
        """, (client_id, phone))
    conn.commit()
    return cursor.fetchall()

def change_client(conn, cursor, id, first_name=None, last_name=None, email=None):
    updates = []
    params = []

    if first_name is not None:
        updates.append("first_name = %s")
        params.append(first_name)
    if last_name is not None:
        updates.append("last_name = %s")
        params.append(last_name)
    if email is not None:
        updates.append("email = %s")
        params.append(email)

    params.append(id)

    query = "UPDATE client SET " + ", ".join(updates) + " WHERE id = %s;"
    cursor.execute(query, params)
    conn.commit()

    return "Успешно"
    
def delete_phone(conn, cursor, id):
    cursor.execute("""
        DELETE FROM client_phone WHERE id=%s;
        """, (id,))
    conn.commit()
    return "Строка удалена"

def delete_client(conn, cursor, id):
    cursor.execute("""
        DELETE FROM client_phone WHERE client_id=%s;
        """, (id,))
    conn.commit()
    cursor.execute("""
        DELETE FROM client WHERE id=%s;
        """, (id,))
    conn.commit()
    return "Строка удалена"

def find_client(conn, cursor, first_name=None, last_name=None, email=None, phone=False):
    selectes = []
    params = []

    if first_name is not None:
        selectes.append("c.first_name = %s")
        params.append(first_name)
    if last_name is not None:
        selectes.append("c.last_name = %s")
        params.append(last_name)
    if email is not None:
        selectes.append("c.email = %s")
        params.append(email)
    
    if phone is True:
        selectes.append("cp.phone IS NULL")
    elif phone:  # phone не False и не True, это строка с номером
        selectes.append("cp.phone = %s")
        params.append(phone)

    conditions= " AND ".join(selectes) 

    query = "SELECT c.id, c.first_name, c.last_name, c.email, cp.phone FROM client c LEFT JOIN client_phone cp ON c.id = cp.client_id WHERE " + conditions
    cursor.execute(query, params)
    return cursor.fetchall()


with psycopg2.connect(database="clients_db", user="postgres", password="postgres") as conn:
    with conn.cursor() as cur:

        #print(create_db(conn, cur))

        #addClient1=add_client(conn, cur, 'Nick', 'Nilson', 'nilson@mail.ru')
        #addClient2=add_client(conn, cur, 'Alba', 'Rozmary', 'rozmary@mail.ru')
        #addClient3=add_client(conn, cur, 'Jack', 'Ket', 'ket@mail.ru')
        #print(addClient1)
        #print(addClient2)
        #print(addClient3)

        #addClientPhone1=add_phone(conn, cur, 1, '89999999999')
        #addClientPhone2=add_phone(conn, cur, 2, '89399393933')
        #addClientPhone3=add_phone(conn, cur, 2, '89399393934')
        #addClientPhone4=add_phone(conn, cur, 2, '89399393935')
        #addClientPhone5=add_phone(conn, cur, 3, None)
        #print(addClientPhone1)
        #print(addClientPhone2)
        #print(addClientPhone3) 
        #print(addClientPhone4) 
        #print(addClientPhone5)

        #changeClient1=change_client(conn, cur, 1, 'Nickael')
        #changeClient2=change_client(conn, cur, 2, 'Rosmary', 4)
        #changeClient3=change_client(conn, cur, 2, last_name='Rosmary')
        #print(changeClient1) 
        #print(changeClient2) 
        #print(changeClient3) 

        #deletePhone1=delete_phone(conn, cur, 2)
        #print(deletePhone1) 

        #deleteClient1=delete_client(conn, cur, 2)
        #print(deleteClient1)  

        #после удаления строк добавила новые строки 

        findClient1=find_client(conn, cur, first_name="Alba", email="rozmary@mail.ru")
        print(findClient1)
        findClient2=find_client(conn, cur, phone=True) #достаем тех, кто не указал номер телефона
        print(findClient2)
        findClient3=find_client(conn, cur, phone="89999999999")
        print(findClient3)

conn.close()

[(4, 'Alba', 'Rosmary', 'rozmary@mail.ru', '89399393933'), (4, 'Alba', 'Rosmary', 'rozmary@mail.ru', '89399393934'), (4, 'Alba', 'Rosmary', 'rozmary@mail.ru', '89399393935')]
[(5, 'Jack', 'Ket', 'ket@mail.ru', None)]
[(1, 'Nickael', 'Nilson', 'nilson@mail.ru', '89999999999')]
